In [ ]:
using Pkg
Pkg.activate("./demo/Project.toml");

In [ ]:
using LinearAlgebra
using Random
using Plots

In [ ]:
function Hamiltonian(N,distribution,J_min,Omega)
    couplings = distribution(N,J_min,Omega)
    H = zeros(Float64, N, N)
    for ii in 1:N-1
        H[ii,ii+1] = couplings[ii]; H[ii+1,ii] = couplings[ii]
    end
    H[1,N] = (-1)^(N+1)*couplings[N]; H[N,1] = (-1)^(N+1)*couplings[N]
    return H
end

function Box_Hamiltonian(N,J_min,Omega)
    return [rand() * (Omega - J_min) + J_min for ii in 1:N]
end

function Binary_Hamiltonian(N,J_min,Omega)
    return [rand([J_min, Omega]) for ii in 1:N]
end

function Diagonalize(A)
    return Diagonal(eigen(A).values), eigen(A).vectors
end;

In [ ]:
function Delta(x)
    if abs(x) < 1e-15
        return 1
    else 
        return 0
    end
end

function Correlation_Fermion(i,j,U,N)
    C = 0.; k_F = Integer(N/2)
    for k in 1:k_F
        C += U[i,k]*U[j,k]
    end
    return C
end
        
function Correlation_Element(i,j,U)
    return Delta(i-j) - 2*Correlation_Fermion(i,j,U,N)
end

function Correlation_Matrix(U,N)
    Corr_Matrix = zeros(Float64,N,N)
    for i in 1:N
        for j in 1:i
            Symm_ij =  Correlation_Element(i,j,U)
            Corr_Matrix[i,j] = Symm_ij
            Corr_Matrix[j,i] = Symm_ij
        end
    end
    return Corr_Matrix
end

function Longitudinal_Correlation(r,N,Corr_Matrix)
    Corr_z = 0.
    for i in 1:N
        j = (i+r-1)%N+1 
        Corr_z += Corr_Matrix[j,j]*Corr_Matrix[i,i] - Corr_Matrix[i,j]^2
    end
    return Corr_z/(4*N)
end

function Transversal_Correlation(r,N,Corr_Matrix)
    Corr_x = 0.
    A = zeros(Float64, r, r)
    for i in 1:N
        for column in 1:r
            for row in 1:r
                ii = (i+column-1)%N+1%N; jj = (i+row-2)%N+1%N
                A[column,row] = Corr_Matrix[ii,jj]
            end
        end
        Corr_x += det(A)
    end
    return -1^r*Corr_x/(4*N)
end

function Correlation_Function(R, Domain, N, samples, distribution, J_min, Omega=1)
    C_zz = zeros(Float64, Domain, samples); C_xx = zeros(Float64, Domain, samples)
    for t in 1:samples
        print()
        H = Hamiltonian(N,distribution,J_min,Omega)
        D , U = Diagonalize(H)
        Corr_Matrix = Correlation_Matrix(U,N)
        for r in 1:Domain
            C_zz[r,t] = Longitudinal_Correlation(R[r],N,Corr_Matrix) 
            C_xx[r,t] = Transversal_Correlation(R[r],N,Corr_Matrix) 
        end
    end
    return C_zz, C_xx
end;

In [ ]:
function Mean(A,n,m)
    mean = zeros(Float64, n, 1)
    for i in 1:m
        mean += A[:,i]
    end
    return mean/m
end
      
function Variance(A,n,m,mean)
    var = zeros(Float64, n, 1)
    for i in 1:m
        var += (A[:,i] - mean).^2
    end
    return var/m
end;

In [ ]:
N = 4000; J_min = 1/4; Omega = 1; samples = 50; distribution = Box_Hamiltonian; seed = 8;

In [ ]:
R = vcat([r for r in 1:2:10],[trunc(Int,10^r) + (trunc(Int,10^r)+ 1) %2  for r in 1:0.06:2.3]) 
Domain = length(R); Random.seed!(seed)
C_zz, C_xx = Correlation_Function(R, Domain, N, samples, distribution, J_min, Omega);

In [ ]:
Mean_C_xx = Mean(C_xx,Domain,samples)
Mean_C_zz = Mean(C_zz,Domain,samples);

In [ ]:
Var_C_xx = Variance(C_xx,Domain,samples,Mean_C_xx)
Var_C_zz = Variance(C_zz,Domain,samples,Mean_C_zz);

In [ ]:
SeriesX = R.^2 .*abs.(Mean_C_xx)
x = 1:2:200; f(x) = 0.14709*x^(3/2); y = f.(x)
plot(x, y, label="Clean system",xaxis=:log,yaxis=:log,color=:orange,)
scatter!(R,SeriesX,xaxis=:log,yaxis=:log,legend=:topleft,label="Box distribution, J_min = 0", 
        markersize=2.2,color=:blue,ylabel=r"$-l^2*|C_{xx}(l)|$",xlabel=r"$l$",ylim=(0.05,300))
hline!([1/12], color=:black, label="1/12",linestyle=:dash)

In [ ]:
scatter(R,Var_C_xx,xaxis=:log,yaxis=:log,legend=:bottomleft,label="Box distribution, J_min = 0", markersize=2.2,color=:green,ylabel="Var_xx",xlabel=r"$l$")

In [ ]:
SeriesZ = -R.^2 .*Mean_C_zz
scatter(R,SeriesZ,xaxis=:log,legend=:topleft,label="Box distribution, J_min = 0", 
markersize=2.2,color=:blue, ylabel=r"$-l^2*C_{zz}(l)$", xlabel=r"$l$",ylims = (0,0.3))
hline!([1/12], color=:black, label="1/12",linestyle=:dash)
hline!([1/pi^2], color=:orange, label="1/π^2")

In [ ]:
scatter(R,Var_C_zz,xaxis=:log,yaxis=:log,legend=:bottomleft,label="Box distribution, J_min = 0", markersize=2.2,color=:green,ylabel="Var_zz",xlabel=r"$l$")